In [50]:
%load_ext autoreload
%autoreload 2

import numpy as np
import json
from glob import glob

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
def remove_a_key(mydict, remove_key, remove_val=""):
    new = []
    for (key, val) in mydict.items():
        if key != remove_key or val != remove_val:
            new.append((key, val))
    return dict(new)

In [55]:
meta = json.load(open("./scripts/001.json"))

In [56]:
flist = meta["filelist"]

In [57]:
# 원천 metadata = 원시 metadata - filelist
mm = dict([(key, val) for (key, val) in meta.items() if key != 'filelist'])

In [58]:
dir_csv = "./scripts/"
for ff in flist:
    tag = ff.split(".mp4")[0].split("_")
    tag = "_".join(tag[1:])

    fn_json = glob(f"../원사이클_산출물/*_rgb_{tag}_????.json")
    fn_json.sort()

    cam = np.genfromtxt(dir_csv + ff.replace(".mp4", "_point.csv"), delimiter=",", skip_header=1)
    point_x = cam[:,2]
    point_y = cam[:,3]
    
    for fn in fn_json:
        anno = json.load(open(fn,"r"))

        frame = int(anno["Annotations"]['image']['id'])
        anno["Metadata"] = mm
        
        # width & height -> int
        anno["Annotations"]["image"]["width"] = int(anno["Annotations"]["image"]["width"])
        anno["Annotations"]["image"]["height"] = int(anno["Annotations"]["image"]["height"])
        
        # REMOVE UPDATED
        anno["Metadata"]['task'] = remove_a_key(anno["Metadata"]['task'], "updated", "")
        anno["Annotations"]['task'] = remove_a_key(anno["Metadata"]['task'], "updated", "")

        # add point to pose
        pose = anno["Annotations"]["pose"]
        pose.update({"point":[int(point_x[frame-1]), int(point_y[frame-1])]})

        with open(fn, "w") as f:
            json.dump(anno, f)#, indent=2)